In [ ]:
from SpecificationMDP import *
from LearningAgents import *
import matplotlib
%matplotlib inline
#matplotlib.rcparams
matplotlib.rcParams['figure.figsize'] = [20,10]

In [ ]:
MDP = CreateDinnerMDP()

In [ ]:
def RecordLearningCurve(MDP, Learner, max_episodes = 10000, steps = 10, temp = 0.01, verbose = False):
    episodes = 0
    mean_rewards = []
    rewards = []
    std_rewards = []
    mean_episode_length = []
    ep = []
    
    for i in range(0,max_episodes,steps):
        
        if verbose:
            print(f'\r Training and evaluating with {i} episodes        ', end = '')
        
        episodes = episodes + steps
        ep.append(episodes)
        Learner.explore(episode_limit = steps, verbose = False)
        evaluator = ExplorerAgent(MDP, input_policy = Learner.create_learned_softmax_policy(temp))
        evaluator.explore(episode_limit = 50)
        r = [record[-1][3] for record in evaluator.episodic_record]
        l = [len(record) for record in evaluator.episodic_record]
        
        rewards.append(r)
        mean_rewards.append(np.mean(r))
        std_rewards.append(np.std(r))
        mean_episode_length.append(np.mean(l))
        
        #print(mean_rewards[-1])
        
        
    return rewards, mean_rewards, std_rewards, mean_episode_length, ep

In [ ]:
import os
import pickle
#os.mkdir('Q_Agent')

In [24]:
def TrainAndTest(LearnerType, MDP, directory, nRuns = 10, episodes = 1000, steps = 1, temp=0.01, verbose=True):
    
    if not os.path.exists(directory):
        os.mkdir(directory)
    for i in range(nRuns):
        print(f'Run: {i} of {nRuns}')
        agent = LearnerType(MDP)
        LCRecord = RecordLearningCurve(MDP, agent, max_episodes=episodes, steps = steps, temp = temp, verbose=verbose)
        with open(os.path.join(directory, f'LC{i}'),'wb') as file:
            pickle.dump({'LC':LCRecord},file)
        del agent

In [ ]:
nRuns = 10
for i in range(nRuns):
    q_agent = QLearningAgent(MDP)
    LCRecords = RecordLearningCurve(MDP, q_agent, max_episodes = 1000, steps = 1, temp = 0.01, verbose = True)
    with open(f'Q_Agent/LC{i}.pkl', 'wb') as file:
        pickle.dump({'LC':LCRecords}, file)
    del q_agent
    #plt.plot(LCRecords[3], LCRecords[0])

In [ ]:
# stoch_eval = ExplorerAgent(MDP, input_policy = q_agent.create_learned_softmax_policy(0.01))
# stoch_eval.explore(episode_limit = 1000)
_ = stoch_eval.visualize_exploration()

Training a single version of the counterfactual agent


Recording the learning curve over multiple runs of the counterfactual agent

In [ ]:
CFAgent = lambda MDP: QLearningAgent(MDP, counterfactual_updates=True)
TrainAndTest(CFAgent, MDP, 'CFAgent', episodes = 500)